In [7]:
from utils import set_env_vars
set_env_vars(path = "/Users/yonatankaplan/dev/llm_predict/env_vars.json")

In [8]:
from agent_creator import create_experts_analyzer_assistant
from asknews_scrapper import AskNewsScrapper
from chat import run_first_stage_forecasters, \
    run_second_stage_forecasters
from config import get_gpt_config
from experts_extractor import run_expert_extractor, expert_creator
from metaculus_api_requests import get_random_question
from PROMPTS import *

In [9]:
#Prompt templates to edit
EXPERTISE_ANALYZER_PROMPT
SPECIFIC_EXPERTISE
NEWS_STEP_INSTRUCTIONS
NEWS_OUTPUT_FORMAT

'\nOutput your response strictly as a JSON object with the following structure:\n{\n  "prior_probability": int,\n  "analysis_updates": [\n    {\n      "forecaster_name": str,\n      "agreement_point": str,\n      "disagreement_point": str,\n      "key_takeaways": List[str],\n      "effect_on_forecast": "+int%" or "-int%"\n    }\n  ],\n  "revised_probability": int\n}\nEnsure the response can be parsed by Python `json.loads`, e.g.: no trailing commas, no single quotes, etc.\n'

In [10]:
# Initialize configs
config = get_gpt_config(42, 0.7, "gpt-4o", 120)

# Create Expert Identifier using EXPERTISE_ANALYZER_PROMPT
expert_identifier = create_experts_analyzer_assistant(config=config, prompt = EXPERTISE_ANALYZER_PROMPT)

# Get Random Question
question = get_random_question()



# upload to metaculus


Row !A156:O156 marked successfully as sampled.


In [11]:
# Run experts identifier using the created prompt
academic_disciplines, frameworks, professional_expertise, specialty = run_expert_extractor(expert_identifier,
                                                                                           question)


In [12]:
# Create an expert for professional and academics using SPECIFIC_EXPERTISE
all_professional_experts = expert_creator(experts=professional_expertise, config=config,
                                          frameworks_specialties=specialty)
all_academic_experts = expert_creator(experts=academic_disciplines, config=config,
                                      frameworks_specialties=frameworks)


OpenAI client config of GPTAssistantAgent(MilitaryIntelligenceBattlefieldAssessmentAgent) - model: gpt-4o
No matching assistant found, creating a new assistant


In [13]:
# Run First Stage Using SPECIFIC_EXPERTISE, prompt = "" will yied the use of the bot's system message
results = run_first_stage_forecasters(all_professional_experts + all_academic_experts,
                                      question['title'] + "\n" + question['description'],prompt = "")

In [14]:
# get news articles
news_scrapper = AskNewsScrapper()
news = news_scrapper._ask_news_engine.news.search_news(query = question['title'] + "\n" + question['description'],
                                                       n_articles=5, return_type="both", strategy="latest news")

In [17]:
# Run Second Stage with prompts NEWS_STEP_INSTRUCTIONS and output format NEWS_OUTPUT_FORMAT
news_analysis_results = run_second_stage_forecasters(all_professional_experts, news, prompt = NEWS_STEP_INSTRUCTIONS,output_format = NEWS_OUTPUT_FORMAT)


In [16]:
# Get mean/final predictions
mean_result_probabilities = [int(result['revised_probability']) for result in news_analysis_results.values()]
print(f"Mean of revised probabilities: {sum(mean_result_probabilities) / len(mean_result_probabilities)}")


Mean of revised probabilities: 8.333333333333334


In [ ]:
# Upload to Metaculus TODO